In [5]:
import matplotlib.pyplot as plt
from subprocess import check_output
import os
input_folder = "../dataset"

###run once
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import graphviz
import pydotplus
from pandas import DataFrame,Series
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
movie_file = pd.read_csv(input_folder+"/movie_metadata_cleaned_filled_values.csv")
movie_data= DataFrame(movie_file)
movie_data.fillna(movie_data.mean())
movie_data = movie_data.dropna()

#movie_data.head(10)
#len(movie_data)

drop_predictors = ['gross', 'worldwide_gross','color', 'movie_title', 'director_name',
       'actor_1_name', 'actor_2_name','actor_3_name', 
       'content_rating', 'genres', 'plot_keywords', 'movie_imdb_link', 'country', 'language', 'release_date','title_year','aspect_ratio']

#define predictors and predicted values
X = movie_data.drop(drop_predictors, axis=1)
y = movie_data.worldwide_gross

from sklearn import preprocessing

####standardize data to address different scales.
standardized_X = preprocessing.scale(X)
standardized_y = preprocessing.scale(y)
#print(standardized_y)

#### split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(standardized_X, standardized_y, train_size=0.7, random_state=0)


In [8]:
n_jobs = 1  # Parallelisation factor for bagging, random forests
random_state = 42
n_estimators = 50
step_factor = 10
axis_step = int(n_estimators/step_factor)

estimators = np.zeros(axis_step)
bagging_mse = np.zeros(axis_step)

# Estimate the Bagging MSE over the full number
# of estimators, across a step size ("step_factor")
for i in range(0, axis_step):
    print("Bagging Estimator: %d of %d..." % (
        step_factor*(i+1), n_estimators)
    )
    bagging = BaggingRegressor(
        DecisionTreeRegressor(), 
        n_estimators=step_factor*(i+1),
        n_jobs=n_jobs,
        random_state=random_state
    )
    bagging.fit(X_train, y_train)
    mse = mean_squared_error(y_test, bagging.predict(X_test))
    estimators[i] = step_factor*(i+1)
    bagging_mse[i] = mse

Bagging Estimator: 10 of 50...
Bagging Estimator: 20 of 50...
Bagging Estimator: 30 of 50...
Bagging Estimator: 40 of 50...
Bagging Estimator: 50 of 50...


In [9]:
print mse

0.365273314353


In [10]:
print bagging_mse

[ 0.3837965   0.36909478  0.3654786   0.36821699  0.36527331]


In [11]:
bagging_mse.mean()

0.37037203827720566

In [12]:
r2_score(y_test, bagging.predict(X_test), multioutput='variance_weighted')

0.68339526004791429